In [ ]:
import os
import sys
from time import sleep

import numpy as np

import GCode
import GRBL


# Inkscape G-Code Spindle Playback

## Test Setup

# Code:

In [25]:
cnc.reset()


In [26]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.laser_mode = 0


ok


In [27]:
print(cnc.laser_mode)


0.0


In [32]:
test_run = GCode.GCode()

test_run.load("/tmp/raf_cut_0010.gcode")

buffer = [line.strip() for line in test_run.buffer if len(line.strip()) > 0]
buffer


['M05 S0',
 'G90',
 'G21',
 'G1 F3000',
 'G1  X45.9189 Y3.6248',
 'G4 P0',
 'M03 S12000',
 'G4 P0',
 'G1 F100.000000',
 'G1  X46.2032 Y3.6248',
 'G1  X46.4875 Y3.3405',
 'G1  X46.4875 Y1.0658',
 'G1  X45.9189 Y0.7815',
 'G1  X46.2032 Y0.7815',
 'G1  X46.7719 Y0.4972',
 'G1  X47.0562 Y0.2129',
 'G4 P0',
 'M05 S0',
 'G1 F3000',
 'G1  X47.0562 Y3.3405',
 'G4 P0',
 'M03 S12000',
 'G4 P0',
 'G1 F100.000000',
 'G1  X47.9092 Y3.6248',
 'G1  X48.4778 Y3.9091',
 'G1  X48.7622 Y4.1934',
 'G1  X49.0465 Y3.9091',
 'G1  X49.6151 Y3.6248',
 'G1  X49.8995 Y3.6248',
 'G1  X49.3308 Y3.3405',
 'G1  X49.3308 Y1.0658',
 'G1  X49.6151 Y0.7815',
 'G1  X49.8995 Y0.7815',
 'G4 P0',
 'M05 S0',
 'G1 F3000',
 'G1  X48.4778 Y3.9091',
 'G4 P0',
 'M03 S12000',
 'G4 P0',
 'G1 F100.000000',
 'G1  X49.0465 Y3.6248',
 'G1  X49.0465 Y2.0034',
 'G1  X49.0465 Y0.7815',
 'G1  X49.3308 Y0.4972',
 'G4 P0',
 'M05 S0',
 'G1 F3000',
 'G1  X47.9092 Y3.6248',
 'G4 P0',
 'M03 S12000',
 'G4 P0',
 'G1 F100.000000',
 'G1  X48.1935 Y3

In [33]:
zero = GCode.GCode()
zero.G92(X=0, Y=0, Z=0)


In [34]:
cnc.run(zero)


0.1037895679473877

In [37]:
cnc.reset()


In [35]:
cnc.run(test_run)


^C


49.26940608024597

In [91]:
prog = GCode.GCode()
prog.G92(X=0, Y=0, Z=0)
for line in buffer:
    line = line.replace(" ", "")
    # print(line)
    if line == "M05S0":
        continue

    elif line in ["G90", "G21"]:
        prog.buffer.append(line)
        continue

    elif line.startswith("G1F3000"):
        prog.G1(F=3000)
        prog.G0(Z=5)
        continue

    elif line.startswith("G1F100"):
        prog.M3(S=12000)
        prog.buffer.append(line)
        prog.G1(Z=-0.1)
        continue

    elif line.startswith("G1X"):
        prog.buffer.append(line)
        continue

    elif line in ["G4P0", "M18", "M03S12000"]:
        continue
    else:
        raise (Exception(line))


In [92]:
prog


<GCode>[cmds=895]

In [93]:
cnc.run(prog)


^C


68.62304067611694

In [94]:
cnc.reset()


In [97]:
def laser2cut(file):
    buffer2 = list()
    laser = GCode.GCode()
    laser.load(file)

    buffer = [line.strip() for line in test_run.buffer if len(line.strip()) > 0]

    prog = GCode.GCode()
    prog.G92(X=0, Y=0, Z=0)
    for line in buffer:
        line = line.replace(" ", "")
        # print(line)
        if line == "M05S0":
            continue

        elif line in ["G90", "G21"]:
            prog.buffer.append(line)
            continue

        elif line.startswith("G1F3000"):
            prog.G1(F=3000)
            prog.G0(Z=5)
            continue

        elif line.startswith("G1F100"):
            prog.M3(S=12000)
            prog.buffer.append(line)
            prog.G1(Z=0)
            continue

        elif line.startswith("G1X"):
            prog.buffer.append(line)
            continue

        elif line in ["G4P0", "M18", "M03S12000"]:
            continue
        else:
            raise (Exception(line))
    return prog


In [98]:
prog = laser2cut("/tmp/cross_cut_0001.gcode")


In [99]:
prog


<GCode>[cmds=895]

In [118]:
cnc.reset()


In [119]:
cnc.run(prog)


SerialException: Could not configure port: (5, 'Input/output error')